In [ ]:
from bs4 import BeautifulSoup

# Obtén el HTML de la página cargada
html_content = browser.page_source

# Analiza el HTML con Beautiful Soup
soup = BeautifulSoup(html_content, "lxml")

import nltk 
import re
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import SnowballStemmer
from nltk.stem import WordNetLemmatizer
import spacy

In [ ]:
nltk.download('stopwords')

porter=SnowballStemmer("english") #elimina prefijos y sufijos sin garantizar que se ana palabra valida
lmtzr = WordNetLemmatizer()  #creo que te devuelve todo a la root form, si que garantiza que sea valida.

STOP_WORDS = set(stopwords.words('english'))

In [ ]:
def preprocess_lower(text):
    """
    Preprocess text by:
       - Converting to lowercase.
       - Removing punctuation.
       - Tokenizing.
       - Removing stopwords.
    
    Returns:
        List[str]: A list of tokens.
    """
    # Lowercase the text
    text_lower = text.lower()

    # Remove punctuation (everything except word characters and whitespace)
    text_no_punct = re.sub(r'[^\w\s]', '', text_lower)

    # Tokenize the cleaned text.
    tokens = word_tokenize(text_no_punct) #divide el texto en palabras individuales

    # Filter out stopwords.
    filtered_tokens = [token for token in tokens if token not in STOP_WORDS]

    return " ".join(filtered_tokens)


def preprocess_stem(text):
    """
    Preprocess text by performing all steps in preprocess_lower() and then applying stemming.
    
    Returns:
        List[str]: A list of stemmed tokens.
    """
    # Get the cleaned lowercased and stopwords-removed tokens.
    joined_tokens = preprocess_lower(text)
    
    tokens=word_tokenize(joined_tokens)

    # Initialize the Porter Stemmer.
    ps = PorterStemmer()

    # Stem each token.
    stemmed_tokens = [ps.stem(token) for token in tokens]

    return " ".join(stemmed_tokens)

In [ ]:
# Initialize a list to store the processed results
processed_descriptions = []

# Loop through the raw descriptions in the `descriptions` list
for desc in descriptions:
    raw_description = desc["Raw Description"] if isinstance(desc, dict) else desc

    if raw_description != "Descripción no encontrada":
        # Apply lowercasing, punctuation removal, and stopword removal
        preprocessed_lower = preprocess_lower(raw_description)

        # Apply stemming
        preprocessed_stem = preprocess_stem(preprocessed_lower)

        # Store the processed results in a dictionary
        processed_descriptions.append({
            "Raw Description": raw_description,
            "Processed (Lowercase, No Stopwords)": preprocessed_lower,
            "Processed (Stemming)": preprocessed_stem
        })
    else:
        # Handle missing descriptions
        processed_descriptions.append({
            "Raw Description": raw_description,
            "Processed (Lowercase, No Stopwords)": "No processing - No description available",
            "Processed (Stemming)": "No processing - No description available"
        })

# Print the processed descriptions
print("\nProcessed Descriptions:")
for i, desc in enumerate(processed_descriptions):
    print(f"Hotel {i + 1}:")
    print(f"  Raw Description: {desc['Raw Description']}")
    print(f"  Processed (Lowercase, No Stopwords): {desc['Processed (Lowercase, No Stopwords)']}")
    print(f"  Processed (Stemming): {desc['Processed (Stemming)']}")

# Optionally, save the processed descriptions to a CSV for further analysis
import pandas as pd
df = pd.DataFrame(processed_descriptions)
df.to_csv("processed_hotel_descriptions.csv", index=False, encoding="utf-8")
print("\nProcessed descriptions saved to 'processed_hotel_descriptions.csv'")
